<a href="https://colab.research.google.com/github/AxelRK200/trend_finder_dataiku/blob/main/Exploration/Donchian_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install ta

In [2]:
import pandas as pd
import yfinance as yf
from ta.volatility import DonchianChannel

# Import et mise en forme #

In [78]:
ndx = yf.Ticker("^NDX")
df_DC = ndx.history(period="max")
# Passe la date d'index à colonne
df_DC.reset_index(inplace=True)

In [79]:
df_DC.drop(columns=['Volume','Dividends','Stock Splits'], inplace=True)

In [80]:
cols_to_round = df_DC.columns.to_list()
print(cols_to_round)

['Date', 'Open', 'High', 'Low', 'Close']


In [81]:
for i in range (1,5):
  df_DC[cols_to_round[i]] = df_DC[cols_to_round[i]].apply(lambda f : round(f,2))

In [82]:
df_DC["Date"] = df_DC.Date.apply(lambda d : d.strftime('%Y-%m-%d'))

# Indicateur technique #

In [83]:
# Stratégie STC + DC10, couplée à DC22 & DC55 -> Turtle Rules
dc_lengths = [10,22,55]

In [84]:
for l in dc_lengths :
  df_DC["DC_up_"+str(l)] = DonchianChannel(high=df_DC.High, low=df_DC.Low, close=df_DC.Close, window=l).donchian_channel_hband()
  df_DC["DC_low_"+str(l)] = DonchianChannel(high=df_DC.High, low=df_DC.Low, close=df_DC.Close, window=l).donchian_channel_lband()
  df_DC["DC_width_"+str(l)] = round( DonchianChannel(high=df_DC.High, low=df_DC.Low, close=df_DC.Close, window=l).donchian_channel_wband(), 2)

In [85]:
df_DC.drop(columns=['High','Open','Low'], inplace=True)
df_DC.dropna(inplace=True)
df_DC.reset_index(drop=True, inplace=True)

In [86]:
df_DC.head()

,Date,Close,DC_up_10,DC_low_10,DC_width_10,DC_up_22,DC_low_22,DC_width_22,DC_up_55,DC_low_55,DC_width_55
0,1985-12-17,131.54,133.18,125.17,6.19,133.18,121.73,9.06,133.18,106.75,22.16
1,1985-12-18,131.14,133.18,126.78,4.93,133.18,121.73,9.03,133.18,106.75,22.10
2,1985-12-19,131.93,133.18,126.78,4.92,133.18,121.74,8.99,133.18,106.75,22.03
3,1985-12-20,132.26,133.18,126.83,4.86,133.18,122.20,8.60,133.18,106.75,21.96
4,1985-12-23,130.73,133.18,127.15,4.61,133.18,122.20,8.58,133.18,106.75,21.89


# Feature Engineering #

In [87]:
df_DC_FE = df_DC.copy()

In [88]:
col_list = df_DC.columns.to_list()
for i in range(2,9,3):
  print(col_list[i])

DC_up_10
DC_up_22
DC_up_55


In [89]:
'''Genère la différence entre les bornes hautes et basses des Donchian Channels.
Détecte les Higher Highs, Lower Highs, Lower Lows et Higher Lows.
Puis la combinaison Higher High & Higher Low, ou Lower Low & Lower High.
Calcule également la différence de largeur des bandes à +2 et +5j'''

for i in range(2,9,3): 
  up_diff = df_DC_FE[col_list[i]] - df_DC_FE[col_list[i]].shift(1)
  df_DC_FE[col_list[i]+"_HH"] = [1 if value >0 else 0 for value in up_diff]
  df_DC_FE[col_list[i]+"_LH"] = [1 if value <0 else 0 for value in up_diff]

  low_diff =  df_DC_FE[col_list[i+1]] - df_DC_FE[col_list[i+1]].shift(1)
  df_DC_FE[col_list[i+1]+"_HL"] = [1 if value >0 else 0 for value in low_diff]
  df_DC_FE[col_list[i+1]+"_LL"] = [1 if value <0 else 0 for value in low_diff]

  all_channels_up = df_DC_FE[col_list[i]+"_HH"] + df_DC_FE[col_list[i+1]+"_HL"]
  df_DC_FE[col_list[i]+"_HHHL"] = [1 if value ==2 else 0 for value in all_channels_up]

  all_channels_down = df_DC_FE[col_list[i]+"_LH"] + df_DC_FE[col_list[i+1]+"_LL"]
  df_DC_FE[col_list[i+1]+"_LHLL"] = [1 if value ==2 else 0 for value in all_channels_down]
  
  df_DC_FE[col_list[i+2]+"_lag2_diff"] = df_DC_FE[col_list[i+2]] - df_DC_FE[col_list[i+2]].shift(2)
  df_DC_FE[col_list[i+2]+"_lag5_diff"] = df_DC_FE[col_list[i+2]] - df_DC_FE[col_list[i+2]].shift(5)

In [90]:
''' Finds if Close breaks upper or lower donchian channel.
Previous DC up/low needs to be taken into consideration, as they are updated at the end of each day.'''

for i in range(2,9,3): 
  
  Close_DCUpLagDiff = df_DC_FE.Close - df_DC_FE[col_list[i]].shift(1)
  df_DC_FE[col_list[i]+"_breakout"] = [1 if d>0 else 0 for d in Close_DCUpLagDiff]

  Close_DCLowLagDiff = df_DC_FE.Close - df_DC_FE[col_list[i+1]].shift(1)
  df_DC_FE[col_list[i+1]+"_breakout"] = [1 if d<0 else 0 for d in Close_DCLowLagDiff]

In [98]:
df_DC_FE.tail()

,Date,Close,DC_up_10,DC_low_10,DC_width_10,DC_up_22,DC_low_22,DC_width_22,DC_up_55,DC_low_55,...,DC_up_55_HHHL,DC_low_55_LHLL,DC_width_55_lag2_diff,DC_width_55_lag5_diff,DC_up_10_breakout,DC_low_10_breakout,DC_up_22_breakout,DC_low_22_breakout,DC_up_55_breakout,DC_low_55_breakout
9358,2023-02-03,12573.36,12880.98,11550.07,10.95,12880.98,10696.42,18.71,12880.98,10671.19,...,0,0,3.61,5.37,0,0,0,0,0,0
9359,2023-02-06,12464.51,12880.98,11550.07,10.90,12880.98,10696.42,18.60,12880.98,10671.19,...,0,0,-0.05,5.37,0,0,0,0,0,0
9360,2023-02-07,12728.27,12880.98,11550.07,10.82,12880.98,10696.42,18.46,12880.98,10671.19,...,0,0,-0.05,5.37,0,0,0,0,0,0
9361,2023-02-08,12495.38,12880.98,11853.37,8.31,12880.98,11050.68,15.38,12880.98,10671.19,...,0,0,-0.05,3.54,0,0,0,0,0,0
9362,2023-02-09,12644.55,12880.98,11906.11,7.85,12880.98,11050.68,15.29,12880.98,10671.19,...,0,0,-0.05,-0.13,0,0,0,0,0,0


In [97]:
df_DC_FE.loc[ df_DC_FE["DC_low_55_LHLL"]==1 ].shape[0]

27

In [99]:
df_DC_FE.columns

Index(['Date', 'Close', 'DC_up_10', 'DC_low_10', 'DC_width_10', 'DC_up_22',
       'DC_low_22', 'DC_width_22', 'DC_up_55', 'DC_low_55', 'DC_width_55',
       'DC_up_10_HH', 'DC_up_10_LH', 'DC_low_10_HL', 'DC_low_10_LL',
       'DC_up_10_HHHL', 'DC_low_10_LHLL', 'DC_width_10_lag2_diff',
       'DC_width_10_lag5_diff', 'DC_up_22_HH', 'DC_up_22_LH', 'DC_low_22_HL',
       'DC_low_22_LL', 'DC_up_22_HHHL', 'DC_low_22_LHLL',
       'DC_width_22_lag2_diff', 'DC_width_22_lag5_diff', 'DC_up_55_HH',
       'DC_up_55_LH', 'DC_low_55_HL', 'DC_low_55_LL', 'DC_up_55_HHHL',
       'DC_low_55_LHLL', 'DC_width_55_lag2_diff', 'DC_width_55_lag5_diff',
       'DC_up_10_breakout', 'DC_low_10_breakout', 'DC_up_22_breakout',
       'DC_low_22_breakout', 'DC_up_55_breakout', 'DC_low_55_breakout'],
      dtype='object')

In [101]:
''' Compte tenu du faible nombre de signaux combinés, on lisse le signal avec une aggrégation,
somme des signaux sur une fenêtre variable. 3j pour DC10, 10j pour DC55'''

df_DC_FE["DC_up_10_HHHL_rolling3"] = df_DC_FE["DC_up_10_HHHL"].rolling(3).sum()
df_DC_FE["DC_low_10_LHLL_rolling3"] = df_DC_FE["DC_low_10_LHLL"].rolling(3).sum()
df_DC_FE["DC_up_22_HHHL_rolling5"] = df_DC_FE["DC_up_22_HHHL"].rolling(5).sum()
df_DC_FE["DC_low_22_LHLL_rolling5"] = df_DC_FE["DC_low_22_LHLL"].rolling(5).sum()
df_DC_FE["DC_up_55_HHHL_rolling10"] = df_DC_FE["DC_up_55_HHHL"].rolling(10).sum()
df_DC_FE["DC_low_55_LHLL_rolling10"] = df_DC_FE["DC_low_55_LHLL"].rolling(10).sum()